# LinkedIn Automated Login with Selenium

This notebook demonstrates how to automate LinkedIn login using Selenium WebDriver.

**Requirements:**
- selenium package installed (`pip install selenium`)
- Chrome browser installed
- ChromeDriver (will be downloaded automatically with recent selenium versions)
- Environment variables set for credentials

## Step 1: Import Required Libraries

In [93]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException
import os
import time
import random

## Step 2: Setup Chrome Driver

This function configures Chrome to appear more like a real browser and avoid detection.

In [94]:
DELAY_MIN = 4.0
DELAY_MAX = 8.0
def random_delay():
    delay = random.uniform(DELAY_MIN, DELAY_MAX)
    time.sleep(delay)

In [95]:
def setup_chrome_driver():
    """
    Initialize Chrome WebDriver with options to appear more like a real browser.

    Returns:
        webdriver.Chrome: Configured Chrome driver instance
    """
    options = webdriver.ChromeOptions()

    # Add options to make the browser appear more human-like
    options.add_argument("--start-maximized")
    options.add_argument("--disable-bots")
    options.add_argument("--disable-automation")
    options.add_argument("--disable-infobars")
    options.add_argument("--disable-blink-features=AutomationControlled")
    
    random_delay()
    # Set a real browser user agent
    user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.0.0 Safari/537.36"
    options.add_argument(f"user-agent={user_agent}")

    # Additional options to avoid detection
    options.add_experimental_option("excludeSwitches", ["enable-automation"])
    options.add_experimental_option('useAutomationExtension', False)

    random_delay()
    # Initialize and return the driver
    driver = webdriver.Chrome(options=options)

    # Remove webdriver property to avoid detection
    driver.execute_script("Object.defineProperty(navigator, 'webdriver', {get: () => undefined})")

    return driver

In [96]:
driver = setup_chrome_driver()

## Step 3: Login to LinkedIn Function

This function handles the entire login process with proper waits and verification.

In [97]:
def login_to_linkedin(driver, email, password):
    """
    Perform automated login to LinkedIn.

    Args:
        driver: Selenium WebDriver instance
        email (str): LinkedIn account email
        password (str): LinkedIn account password

    Returns:
        bool: True if login successful, False otherwise
    """
    try:
        # Navigate to LinkedIn login page
        print("Navigating to LinkedIn login page...")
        driver.get("https://www.linkedin.com/login")

        # Wait for the login page to load
        wait = WebDriverWait(driver, 10)

        random_delay()
        # Wait for and find the email field
        print("Waiting for login form to load...")
        email_field = wait.until(
            EC.presence_of_element_located((By.ID, "username"))
        )

        random_delay()
        # Fill in the email
        print("Entering email...")
        email_field.clear()
        email_field.send_keys(email)
        random_delay()  # Small delay to mimic human behavior

        # Find and fill the password field
        password_field = driver.find_element(By.ID, "password")
        print("Entering password...")
        password_field.clear()
        password_field.send_keys(password)
        random_delay()

        # Find and click the sign in button
        print("Clicking sign in button...")
        sign_in_button = driver.find_element(
            By.XPATH, "//button[@type='submit']"
        )
        sign_in_button.click()

        # Wait for redirect to feed/home page
        print("Waiting for login to complete...")
        random_delay()  # Give time for redirect

        # Verify login success
        current_url = driver.current_url

        # Check for successful login indicators
        if '/feed/' in current_url or '/mynetwork/' in current_url or '/jobs/' in current_url:
            print("Login successful - redirected to authenticated page!")
            return True

        # Alternative check: Look for navigation elements that only appear when logged in
        try:
            wait.until(
                EC.presence_of_element_located((By.CLASS_NAME, "global-nav"))
            )
            print("Login successful - navigation bar detected!")
            return True
        except TimeoutException:
            # Check if there's an error message or we're still on login page
            if '/login' in driver.current_url:
                print("Login failed - still on login page")
                # Try to capture error message if present
                try:
                    error_element = driver.find_element(By.ID, "error-for-password")
                    print(f"Error message: {error_element.text}")
                except NoSuchElementException:
                    pass
                return False
            else:
                # If we're not on login page but didn't find nav, still might be successful
                print("Login may have succeeded - not on login page anymore")
                return True

    except TimeoutException:
        print("Timeout while waiting for page elements")
        return False
    except NoSuchElementException as e:
        print(f"Could not find required element: {e}")
        return False
    except Exception as e:
        print(f"Error during login: {e}")
        return False

## Step 4: Main Session Function

This function orchestrates the entire process: loading credentials, setting up the driver, and logging in.

In [112]:
def open_linkedin_session():
    """
    Main function to open a LinkedIn session with automated login.
    Loads credentials from environment variables and returns an active driver.

    Returns:
        webdriver.Chrome: Active Chrome driver instance logged into LinkedIn
        None: If login fails or credentials are missing
    """
    try:
        #
        # Load credentials from environment variables
        # never load and keep creds in stack, always query from
        # env when needed
        #
        print("Loading credentials from environment variables...")
        isEmail = bool(os.getenv('LINKEDIN_EMAIL'))
        isPwd = bool(os.getenv('LINKEDIN_EMAIL'))

        # Validate credentials
        if not isEmail or not isPwd:
            print("Error: LINKEDIN_EMAIL and LINKEDIN_PASSWORD environment variables must be set")
            print("Set them using:")
            print('  export LINKEDIN_EMAIL="your@email.com"')
            print('  export LINKEDIN_PASSWORD="yourpassword"')
            return None

        print(f"Credentials loaded.")

        # Setup Chrome driver
        print("Setting up Chrome driver...")
        driver = setup_chrome_driver()

        # Login to LinkedIn
        print("Attempting to login to LinkedIn...")
        login_success = login_to_linkedin(driver, 
                                          os.getenv('LINKEDIN_EMAIL'), 
                                          os.getenv('LINKEDIN_PASSWORD'))

        if login_success:
            print("\n" + "="*50)
            print("Successfully logged into LinkedIn!")
            print(f"Current URL: {driver.current_url}")
            print("="*50 + "\n")
            return driver
        else:
            print("\nLogin failed. Closing driver...")
            driver.quit()
            return None

    except Exception as e:
        print(f"Error in open_linkedin_session: {e}")
        if 'driver' in locals():
            driver.quit()
        return None

## Step 5: Set Environment Variables

**Load from .env file (recommended)**

Create a `.env` file in the project root with:
```
LINKEDIN_EMAIL=your@email.com
LINKEDIN_PASSWORD=yourpassword
```

In [100]:
# OPTION 2: Load from .env file (recommended)
# First install: pip install python-dotenv

from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Verify they're loaded (will show True/False without revealing the actual values)
print(f"LINKEDIN_EMAIL set: {bool(os.getenv('LINKEDIN_EMAIL'))}")
print(f"LINKEDIN_PASSWORD set: {bool(os.getenv('LINKEDIN_PASSWORD'))}")

LINKEDIN_EMAIL set: True
LINKEDIN_PASSWORD set: True


## Step 7: LinkedIn Jobs Search Functions

These functions handle navigating to the Jobs page and interacting with search boxes.

In [101]:
def navigate_to_jobs_page(driver):
    """
    Navigate to LinkedIn Jobs page and wait for it to load.
    
    Args:
        driver: Selenium WebDriver instance
        
    Returns:
        bool: True if navigation successful, False otherwise
    """
    try:
        print("Navigating to LinkedIn Jobs page...")
        driver.get("https://www.linkedin.com/jobs/")
        
        # Wait for page to load - look for the jobs search container
        wait = WebDriverWait(driver, 10)
        
        random_delay()
        # Wait for either the search container or jobs content to appear
        try:
            # wait.until(
            #     EC.presence_of_element_located((By.CSS_SELECTOR, ".jobs-search-box, .jobs-home, [class*='jobs-search']"))
            # )
            wait.until(
                EC.presence_of_element_located((By.CSS_SELECTOR, "[placeholder='Title, skill or Company']"))
            )
            print("Jobs page loaded successfully!")
            return True
        except TimeoutException:
            print("Jobs page elements not found, but continuing...")
            return True  # Still return True as we navigated to the URL
            
    except Exception as e:
        print(f"Error navigating to jobs page: {e}")
        return False

In [102]:
# Open LinkedIn session
driver = open_linkedin_session()

if driver:
    print("\nSession is active! You can now interact with LinkedIn.")
    print(f"Current page: {driver.current_url}")
else:
    print("\nFailed to establish LinkedIn session.")

Loading credentials from environment variables...
Credentials loaded.
Setting up Chrome driver...
Attempting to login to LinkedIn...
Navigating to LinkedIn login page...
Waiting for login form to load...
Entering email...
Entering password...
Clicking sign in button...
Waiting for login to complete...
Login successful - redirected to authenticated page!

Successfully logged into LinkedIn!
Current URL: https://www.linkedin.com/feed/


Session is active! You can now interact with LinkedIn.
Current page: https://www.linkedin.com/feed/


In [104]:
def find_and_fill_search_boxes(driver, job_query, location_query):
    """
    Find and fill LinkedIn job search boxes using specific placeholder text.
    Searches for boxes with placeholders:
    1. "Title, skill or Company" 
    2. "City, state, or zip code"
    
    Args:
        driver: Selenium WebDriver instance
        job_query (str): Job title/keyword to search for
        location_query (str): Location to search in
        
    Returns:
        tuple: (success: bool, message: str)
    """
    try:
        wait = WebDriverWait(driver, 10)

        job_title_input = None
        location_input = None

        random_delay()

        # Wait for and fill the job title input
        job_title_input = wait.until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "[placeholder='Title, skill or Company']"))
        )
        # Check if we found both boxes
        if not job_title_input:
            return (False, "Job search box not found")

        print(f"\nFilling search boxes...")
        print(f"  Job query: '{job_query}'")

        # job_title_input.clear()
        # driver.execute_script("arguments[0].value = '';", job_title_input)
        # random_delay()
        job_title_input.click()
        job_title_input.send_keys(Keys.CONTROL + "a")
        job_title_input.send_keys(job_query)

        random_delay()

        # Wait for and fill the location input
        location_input = wait.until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "[placeholder='City, state, or zip code']"))
        )
        if not location_input:
            return (False, "Location box not found")

        print(f"  Location query: '{location_query}'")


        location_input.click()
        location_input.send_keys(Keys.CONTROL + "a")
        location_input.send_keys(location_query)
        time.sleep(1)  # Wait for dropdown
        location_input.send_keys(Keys.ARROW_DOWN)  # Select first suggestion
        # Press Enter to submit the search
        print("  Pressing Enter to submit search...")
        location_input.send_keys(Keys.RETURN)

        # random_delay()

        # # Wait for and fill the location input
        # location_input = wait.until(
        #     EC.presence_of_element_located((By.CSS_SELECTOR, "[placeholder='City, state, or zip code']"))
        # )
        # location_input.send_keys(Keys.RETURN)
        # random_delay()  # Wait for search results to load
        
        print("  ✓ Search submitted successfully!")
                        
        return (True, "Success")
        
    except Exception as e:
        return (False, f"Error: {str(e)}")

In [113]:
# Open LinkedIn session
driver = open_linkedin_session()

if driver:
    print("\nSession is active! You can now interact with LinkedIn.")
    print(f"Current page: {driver.current_url}")
else:
    print("\nFailed to establish LinkedIn session.")
navigate_to_jobs_page(driver)
find_and_fill_search_boxes(driver, "ML Engineer", "New York")

Loading credentials from environment variables...
Credentials loaded.
Setting up Chrome driver...
Attempting to login to LinkedIn...
Navigating to LinkedIn login page...
Waiting for login form to load...
Entering email...
Entering password...
Clicking sign in button...
Waiting for login to complete...
Login successful - redirected to authenticated page!

Successfully logged into LinkedIn!
Current URL: https://www.linkedin.com/feed/


Session is active! You can now interact with LinkedIn.
Current page: https://www.linkedin.com/feed/
Navigating to LinkedIn Jobs page...
Jobs page loaded successfully!

Filling search boxes...
  Job query: 'ML Engineer'
  Location query: 'New York'
  Pressing Enter to submit search...
  ✓ Search submitted successfully!


(True, 'Success')

In [ ]:
driver.page_source

In [107]:
rendered_html = driver.execute_script("return document.body.innerHTML")
# Or for full page:
full_rendered_html = driver.execute_script("return document.documentElement.outerHTML")

In [116]:
wait = WebDriverWait(driver, 10)

# Find all job cards
job_cards = driver.find_elements(By.CSS_SELECTOR, "div.job-card-container")

for card in job_cards:
    try:
        random_delay()
        # Title
        title_elem = card.find_element(By.CSS_SELECTOR, "a.job-card-list__title")
        title = title_elem.text.strip()
        job_url = title_elem.get_attribute("href")
        
        # Company
        company = card.find_element(By.CSS_SELECTOR, "span.job-card-container__primary-description").text.strip()
        
        # Location
        location = card.find_element(By.CSS_SELECTOR, "li.job-card-container__metadata-item").text.strip()
        
        # Posted time (if available)
        time_posted = card.find_element(By.CSS_SELECTOR, "time").get_attribute("datetime")
        
        print(f"Title: {title}")
        print(f"Company: {company}")
        print(f"Location: {location}")
        print(f"Posted: {time_posted}")
        print(f"URL: {job_url}")
        print("-" * 50)
        
    except Exception as e:
        print(f"Error: {e}")
        continue

In [118]:
# Save what Selenium sees
with open('selenium_source.html', 'w', encoding='utf-8') as f:
    f.write(driver.page_source)

# Also save rendered version
with open('selenium_rendered.html', 'w', encoding='utf-8') as f:
    f.write(driver.execute_script("return document.documentElement.outerHTML"))

In [119]:
# Try different selectors
all_divs = driver.find_elements(By.TAG_NAME, "div")
print(f"Total divs: {len(all_divs)}")

all_lis = driver.find_elements(By.TAG_NAME, "li")
print(f"Total li elements: {len(all_lis)}")

# Try partial class match
job_elements = driver.find_elements(By.CSS_SELECTOR, "[class*='job-card']")
print(f"Elements with 'job-card' in class: {len(job_elements)}")

Total divs: 52
Total li elements: 3
Elements with 'job-card' in class: 0


In [120]:
print(f"Current URL: {driver.current_url}")
print(f"Page title: {driver.title}")

Current URL: https://www.linkedin.com/jobs/search/?currentJobId=4318177855&geoId=105080838&keywords=ML%20Engineer&origin=JOBS_HOME_LOCATION_AUTOCOMPLETE
Page title: (2) ML Engineer Jobs | LinkedIn


In [121]:
def debug_job_elements(driver):
    """
    Debug what job-related elements are on the page.
    """
    # Check different possible selectors
    selectors = [
        "div.job-card-container",
        "li.jobs-search-results__list-item",
        "div[class*='job-card']",
        "li[class*='job']",
        "[data-occludable-job-id]",
        "div.jobs-search-results-list",
        "ul.jobs-search__results-list"
    ]
    
    print("\n" + "="*80)
    print("Job Element Detection:")
    print("="*80 + "\n")
    
    for selector in selectors:
        elements = driver.find_elements(By.CSS_SELECTOR, selector)
        print(f"{selector:50} -> {len(elements)} found")
    
    # Check if jobs are in an iframe
    iframes = driver.find_elements(By.TAG_NAME, "iframe")
    print(f"\nIframes on page: {len(iframes)}")
    
    # Try to find any element with 'job' in the class
    all_with_job = driver.find_elements(By.XPATH, "//*[contains(@class, 'job')]")
    print(f"Elements with 'job' in class: {len(all_with_job)}")
    
    if all_with_job:
        print("\nFirst few 'job' elements:")
        for elem in all_with_job[:5]:
            print(f"  Tag: {elem.tag_name}, Class: {elem.get_attribute('class')[:100]}")

# Run it
debug_job_elements(driver)


Job Element Detection:

div.job-card-container                             -> 0 found
li.jobs-search-results__list-item                  -> 0 found
div[class*='job-card']                             -> 0 found
li[class*='job']                                   -> 0 found
[data-occludable-job-id]                           -> 0 found
div.jobs-search-results-list                       -> 0 found
ul.jobs-search__results-list                       -> 0 found

Iframes on page: 1
Elements with 'job' in class: 0


In [122]:
# Switch to the iframe
iframes = driver.find_elements(By.TAG_NAME, "iframe")
if iframes:
    print(f"Switching to iframe...")
    driver.switch_to.frame(iframes[0])
    
    # Now try finding job elements
    job_cards = driver.find_elements(By.CSS_SELECTOR, "div.job-card-container")
    print(f"Found {len(job_cards)} job cards in iframe")
    
    # Extract job data
    for card in job_cards:
        try:
            random_delay()
            title = card.find_element(By.CSS_SELECTOR, "a.job-card-list__title").text
            company = card.find_element(By.CSS_SELECTOR, "span.job-card-container__primary-description").text
            location = card.find_element(By.CSS_SELECTOR, "li.job-card-container__metadata-item").text
            
            print(f"\nTitle: {title}")
            print(f"Company: {company}")
            print(f"Location: {location}")
        except Exception as e:
            print(f"Error: {e}")
    
    # Switch back to main content when done
    driver.switch_to.default_content()

Switching to iframe...
Found 7 job cards in iframe
Error: Message: no such element: Unable to locate element: {"method":"css selector","selector":"a.job-card-list__title"}
  (Session info: chrome=141.0.7390.122); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#nosuchelementexception
Stacktrace:
0   chromedriver                        0x000000010290fcf8 cxxbridge1$str$ptr + 2895872
1   chromedriver                        0x0000000102907c34 cxxbridge1$str$ptr + 2862908
2   chromedriver                        0x000000010242d570 _RNvCs47EqcsrPRmA_7___rustc35___rust_no_alloc_shim_is_unstable_v2 + 74324
3   chromedriver                        0x0000000102474f34 _RNvCs47EqcsrPRmA_7___rustc35___rust_no_alloc_shim_is_unstable_v2 + 367640
4   chromedriver                        0x000000010246a6cc _RNvCs47EqcsrPRmA_7___rustc35___rust_no_alloc_shim_is_unstable_v2 + 324528
5   chromedriver                        0x00000001024b63

In [125]:
# Switch to iframe
driver.switch_to.frame(driver.find_elements(By.TAG_NAME, "iframe")[0])

# Find job cards
job_cards = driver.find_elements(By.CSS_SELECTOR, "div.job-card-container")
print(f"Found {len(job_cards)} job cards")

# Check what's inside the first card
if job_cards:
    first_card = job_cards[0]
    print("\nFirst card HTML:")
    print(first_card.get_attribute("outerHTML")[:500])  # First 500 chars
    
    # Try finding any <a> tag
    links = first_card.find_elements(By.TAG_NAME, "a")
    print(f"\nLinks in first card: {len(links)}")
    
    # Try finding any span
    spans = first_card.find_elements(By.TAG_NAME, "span")
    print(f"Spans in first card: {len(spans)}")
    
    # Print all text in the card
    print(f"\nAll text in card:\n{first_card.text}")

# Switch back
driver.switch_to.default_content()

Found 11 job cards

First card HTML:
<div data-job-id="4318177855" class="display-flex job-card-container relative job-card-list
        job-card-container--clickable
        
        job-card-list--underline-title-on-hover jobs-search-results-list__list-item--active jobs-search-two-pane__job-card-container--viewport-tracking-0" aria-current="page">

      <div>
        <div id="ember150" class="job-card-list__entity-lockup  artdeco-entity-lockup artdeco-entity-lockup--size-4 ember-view">
            <div id="ember151" class="job-c

Links in first card: 1
Spans in first card: 12

All text in card:
ML Engineer Summer Intern (Remote & Paid)
ML Engineer Summer Intern (Remote & Paid) with verification
Experian
United States (Remote)
401(k) benefit
Viewed
Promoted
Easy Apply


In [126]:
# Switch to iframe
driver.switch_to.frame(driver.find_elements(By.TAG_NAME, "iframe")[0])

# Check for nested iframes
nested_iframes = driver.find_elements(By.TAG_NAME, "iframe")
print(f"Nested iframes: {len(nested_iframes)}")

# Look for job details container
selectors = [
    "div.jobs-search__job-details",
    "div.jobs-details",
    "div[class*='job-details']",
    "div.scaffold-layout__detail",
    "section[class*='details']"
]

for selector in selectors:
    elements = driver.find_elements(By.CSS_SELECTOR, selector)
    print(f"{selector:45} -> {len(elements)} found")

# Click the first job card to see if details load
job_cards = driver.find_elements(By.CSS_SELECTOR, "div.job-card-container")
if job_cards:
    print("\nClicking first job card...")
    job_cards[0].click()
    time.sleep(2)  # Wait for details to load
    
    # Check again for details
    print("\nAfter clicking:")
    for selector in selectors:
        elements = driver.find_elements(By.CSS_SELECTOR, selector)
        print(f"{selector:45} -> {len(elements)} found")

driver.switch_to.default_content()

Nested iframes: 9
div.jobs-search__job-details                  -> 1 found
div.jobs-details                              -> 1 found
div[class*='job-details']                     -> 35 found
div.scaffold-layout__detail                   -> 1 found
section[class*='details']                     -> 1 found

Clicking first job card...

After clicking:
div.jobs-search__job-details                  -> 1 found
div.jobs-details                              -> 1 found
div[class*='job-details']                     -> 35 found
div.scaffold-layout__detail                   -> 1 found
section[class*='details']                     -> 1 found


In [ ]:
# Switch to iframe. iframe[0] contains the job listings. 
driver.switch_to.frame(driver.find_elements(By.TAG_NAME, "iframe")[0])

# Click first job
job_cards = driver.find_elements(By.CSS_SELECTOR, "div.job-card-container")
print("Length of jobcards = ", len(job_cards))
job_cards[2].click()
time.sleep(2)

# Get the details panel
details_panel = driver.find_element(By.CSS_SELECTOR, "div.jobs-search__job-details")

# Print the structure
print("Details panel HTML (first 100 chars):")
print(details_panel.get_attribute("innerHTML")[:100])

# Find all links in details
links = details_panel.find_elements(By.TAG_NAME, "a")
print(f"\nLinks in details: {len(links)}")
for i, link in enumerate(links[:5]):
    print(f"  Link {i}: {link.text[:50]}")

# Find all divs
divs = details_panel.find_elements(By.TAG_NAME, "div")
print(f"\nDivs in details: {len(divs)}")

# Print all visible text
print(f"\nAll text in details panel (first 500 chars):\n{details_panel.text[:500]}")

# switches out of iframe
driver.switch_to.default_content() 


# parse next jobcard
job_cards = driver.find_elements(By.CSS_SELECTOR, "div.job-card-container")
print("Length of jobcards = ", len(job_cards))
job_cards[1].click()
time.sleep(2)

# Get the details panel
details_panel = driver.find_element(By.CSS_SELECTOR, "div.jobs-search__job-details")

# Print the structure
print("Details panel HTML (first 100 chars):")
print(details_panel.get_attribute("innerHTML")[:100])

# Find all links in details
links = details_panel.find_elements(By.TAG_NAME, "a")
print(f"\nLinks in details: {len(links)}")
for i, link in enumerate(links[:5]):
    print(f"  Link {i}: {link.text[:50]}")

# Find all divs
divs = details_panel.find_elements(By.TAG_NAME, "div")
print(f"\nDivs in details: {len(divs)}")

# Print all visible text
print(f"\nAll text in details panel (first 500 chars):\n{details_panel.text[:500]}")

driver.switch_to.default_content()

Length of jobcards =  7
Details panel HTML (first 100 chars):

          
              
    <!---->
    <div class="jobs-search__job-details--wrapper">
      <di

Links in details: 14
  Link 0: 
  Link 1: Pinterest
  Link 2: Software Engineer Intern 2026 — Remote (US)
  Link 3: 
  Link 4: Try Premium for $0

Divs in details: 94

All text in details panel (first 500 chars):
Pinterest
Share
Show more options
Software Engineer Intern 2026 — Remote (US) 
United States · Reposted 1 week ago · Over 100 people clicked apply
Promoted by hirer · Responses managed off LinkedIn
$8,250/month - $11K/month
Remote
Internship
Apply
Save
Save Software Engineer Intern 2026 — Remote (US)  at Pinterest
Your profile is missing required qualifications
Show match details
BETA
Is this information helpful?
Get personalized tips to stand out to hirers
Find jobs where you’re a top applicant
Length of jobcards =  0


IndexError: list index out of range

In [ ]:
# Switch to iframe
driver.switch_to.frame(driver.find_elements(By.TAG_NAME, "iframe")[0])

# Click first job
job_cards = driver.find_elements(By.CSS_SELECTOR, "div.job-card-container")
job_cards[0].click()
time.sleep(2)

# Get details panel
details_panel = driver.find_element(By.CSS_SELECTOR, "div.jobs-search__job-details")

# Get all links
links = details_panel.find_elements(By.TAG_NAME, "a")

print(f"Total links: {len(links)}\n")
print("="*80)

for i, link in enumerate(links):
    try:
        text = link.text.strip()
        href = link.get_attribute("href")
        aria_label = link.get_attribute("aria-label")
        classes = link.get_attribute("class")
        
        print(f"\nLink #{i}:")
        print(f"  Text: {text[:100]}")
        print(f"  Aria-label: {aria_label[:100] if aria_label else 'None'}")
        print(f"  Href: {href[:100] if href else 'None'}")
        print(f"  Classes: {classes[:80] if classes else 'None'}")
    except Exception as e:
        print(f"  Error: {e}")

print("\n" + "="*80)

driver.switch_to.default_content()

In [ ]:
def save_page_source(driver, filename="linkedin_jobs_page.html"):
    """
    Save page source to file and extract all input element details for debugging.
    
    Args:
        driver: Selenium WebDriver instance
        filename (str): Name of file to save HTML to
        
    Returns:
        str: Filename where page source was saved
    """
    try:
        # Get page source
        page_source = driver.page_source
        
        # Save to file
        with open(filename, 'w', encoding='utf-8') as f:
            f.write(page_source)
        print(f"\n✓ Page source saved to: {filename}")
        
        # Find all input elements
        input_elements = driver.find_elements(By.TAG_NAME, "input")
        
        print(f"\n{'='*80}")
        print(f"Found {len(input_elements)} input elements on the page:")
        print(f"{'='*80}\n")
        
        # Extract and print details of each input
        for i, element in enumerate(input_elements, 1):
            try:
                elem_id = element.get_attribute("id") or "(no id)"
                elem_class = element.get_attribute("class") or "(no class)"
                elem_placeholder = element.get_attribute("placeholder") or "(no placeholder)"
                elem_aria_label = element.get_attribute("aria-label") or "(no aria-label)"
                elem_type = element.get_attribute("type") or "(no type)"
                elem_name = element.get_attribute("name") or "(no name)"
                
                print(f"Input #{i}:")
                print(f"  ID:           {elem_id}")
                print(f"  Class:        {elem_class}")
                print(f"  Type:         {elem_type}")
                print(f"  Name:         {elem_name}")
                print(f"  Placeholder:  {elem_placeholder}")
                print(f"  Aria-label:   {elem_aria_label}")
                print()
                
            except Exception as e:
                print(f"  Error reading element {i}: {e}\n")
        
        # Also save input details to a separate file
        details_filename = filename.replace('.html', '_inputs.txt')
        with open(details_filename, 'w', encoding='utf-8') as f:
            f.write(f"Input Elements Analysis\n")
            f.write(f"{'='*80}\n\n")
            
            for i, element in enumerate(input_elements, 1):
                try:
                    f.write(f"Input #{i}:\n")
                    f.write(f"  ID:           {element.get_attribute('id')}\n")
                    f.write(f"  Class:        {element.get_attribute('class')}\n")
                    f.write(f"  Type:         {element.get_attribute('type')}\n")
                    f.write(f"  Name:         {element.get_attribute('name')}\n")
                    f.write(f"  Placeholder:  {element.get_attribute('placeholder')}\n")
                    f.write(f"  Aria-label:   {element.get_attribute('aria-label')}\n")
                    f.write(f"\n")
                except Exception as e:
                    f.write(f"  Error: {e}\n\n")
        
        print(f"✓ Input element details saved to: {details_filename}")
        
        return filename
        
    except Exception as e:
        print(f"Error saving page source: {e}")
        return None

In [147]:
driver.switch_to.default_content()

# Switch to iframe
driver.switch_to.frame(driver.find_elements(By.TAG_NAME, "iframe")[0])

texts = []
# Click first job
job_cards = driver.find_elements(By.CSS_SELECTOR, "div.job-card-container")
for i in range(len(job_cards)):
    job_cards[i].click()
    time.sleep(2)

    # Get the job description article
    try:
        job_description = driver.find_element(By.CSS_SELECTOR, "article.jobs-description__container")
        full_description = job_description.text
        
        print("Full Job Description:")
        print("="*80)
        print(full_description)
        print("="*80)
        
        # # Or get specific sections
        # sections = job_description.find_elements(By.TAG_NAME, "p")
        # print(f"\nFound {len(sections)} paragraphs")
        
        # # Or get the inner HTML if you need to preserve structure
        # description_html = job_description.get_attribute("innerHTML")
        texts.append(full_description)
        
    except Exception as e:
        print(f"Error: {e}")

driver.switch_to.default_content()

Full Job Description:
About the job
Job Posting - Salary Range: See Pay Range

Company Description

Experian is a global data and technology company, powering opportunities for people and businesses around the world. We help redefine lending practices, uncover and prevent fraud, simplify healthcare, create digital marketing solutions, and gain deeper insights into the automotive market, all using our unique combination of data, analytics, and software. We also assist millions of people in realizing their financial goals and saving time and money.

We operate across a range of markets, from financial services to healthcare, automotive, agrifinance, insurance, and many more industry segments.

We invest in people and new advanced technologies to unlock the power of data and to innovate. We're focused on powering opportunities, which is why the Experian Summer Internship Program gives students across the country the chance to apply their education to real-world challenges through meaningf

In [ ]:
import pandas as pd
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

JOBS_IFRAME_INDEX = 0

def extract_linkedin_jobs(driver, max_jobs=10, output_file="linkedin_jobs.csv"):
    """
    Extract job listings from LinkedIn jobs page and save to CSV.
    
    Args:
        driver: Selenium WebDriver instance (should be on LinkedIn jobs page)
        max_jobs (int): Maximum number of jobs to extract
        output_file (str): CSV filename to save results
        
    Returns:
        tuple: (DataFrame, success_count, error_count)
    """
    jobs_data = []
    success_count = 0
    error_count = 0
    
    try:
        # Switch to main jobs iframe
        print("Switching to jobs iframe...")
        iframes = driver.find_elements(By.TAG_NAME, "iframe")
        if not iframes:
            print("No iframe found!")
            return None, 0, 0
        print(f"Found {len(iframes)} iframes on the page.")
            
        driver.switch_to.frame(iframes[JOBS_IFRAME_INDEX])
        
        # Get all job cards
        job_cards = driver.find_elements(By.CSS_SELECTOR, "div.job-card-container")
        total_jobs = min(len(job_cards), max_jobs)
        texts = []
        print(f"\nFound {len(job_cards)} job cards. Extracting {total_jobs} jobs...")
        print("="*80)

        for i in range(total_jobs):
            job_cards[i].click()
            time.sleep(2) # Wait for details to load

            # Get the job description article
            try:
                # # Get details panel
                details_panel = driver.find_element(By.CSS_SELECTOR, "div.jobs-search__job-details")

                links = details_panel.find_elements(By.TAG_NAME, "a")
                company = links[1].text.strip() if len(links) > 1 else "N/A"
                title = links[2].text.strip() if len(links) > 2 else "N/A"

                print("links:", links)
                
                if not company or not title:
                    print("Company({company}) or Title({title}) not found for job_card[{i}]: {card}")

                random_delay()
                
                # Extract full job description
                try:
                    job_description_elem = details_panel.find_element(By.CSS_SELECTOR, "article.jobs-description__container")
                    description = job_description_elem.text.strip()
                    print("Job Description (first 200 chars):")
                    print("="*80)
                    print(description[:200])  # Print first 200 chars
                    print("="*80)
                except:
                    description = "Description not available"
                    print("***** Warning: Job description not found. Job: {company} - {title}")
                
                texts.append(description)
                
                # Store job data
                job_info = {
                    'company': company,
                    'title': title,
                    'description': description,
                    'extraction_timestamp': pd.Timestamp.now()
                }

                jobs_data.append(job_info)
                success_count += 1
                
                print(f"✓ Job {i}/{total_jobs}: {title} at {company}")
                random_delay() # humanize
                
            except Exception as e:
                error_count += 1
                print(f"✗ Job {i}/{total_jobs}: Error - {str(e)[:100]}")
                continue        
        
        # Switch back to main content
        driver.switch_to.default_content()
        
        # Create DataFrame
        if jobs_data:
            df = pd.DataFrame(jobs_data)
            
            # Save to CSV
            df.to_csv(output_file, index=False)
            print(f"\n{'='*80}")
            print(f"✓ Successfully saved {len(df)} jobs to '{output_file}'")
            print(f"  Success: {success_count} | Errors: {error_count}")
            print(f"{'='*80}")
            
            return df, success_count, error_count
        else:
            print("No jobs extracted!")
            return None, 0, error_count
            
    except Exception as e:
        print(f"Fatal error: {e}")
        driver.switch_to.default_content()
        return None, success_count, error_count

# Usage:
# df, success, errors = extract_linkedin_jobs(driver, max_jobs=20, output_file="my_jobs.csv")

In [149]:
driver.switch_to.default_content()

In [150]:
extract_linkedin_jobs(driver, max_jobs=10, output_file="linkedin_jobs.csv")

Switching to jobs iframe...
Found 2 iframes on the page.

Found 11 job cards. Extracting 10 jobs...
links: [<selenium.webdriver.remote.webelement.WebElement (session="24eeb215b080ecc873b54139d2818707", element="f.9B8ED41C6F9715A95AFF930F0353E53E.d.123018A32A725E50944654232DACC69F.e.155")>, <selenium.webdriver.remote.webelement.WebElement (session="24eeb215b080ecc873b54139d2818707", element="f.9B8ED41C6F9715A95AFF930F0353E53E.d.123018A32A725E50944654232DACC69F.e.843")>, <selenium.webdriver.remote.webelement.WebElement (session="24eeb215b080ecc873b54139d2818707", element="f.9B8ED41C6F9715A95AFF930F0353E53E.d.123018A32A725E50944654232DACC69F.e.157")>, <selenium.webdriver.remote.webelement.WebElement (session="24eeb215b080ecc873b54139d2818707", element="f.9B8ED41C6F9715A95AFF930F0353E53E.d.123018A32A725E50944654232DACC69F.e.158")>, <selenium.webdriver.remote.webelement.WebElement (session="24eeb215b080ecc873b54139d2818707", element="f.9B8ED41C6F9715A95AFF930F0353E53E.d.123018A32A725E509446

(              company                                              title  \
 0            Experian          ML Engineer Summer Intern (Remote & Paid)   
 1              Oracle                               Software Developer 1   
 2           Pinterest        Software Engineer Intern 2026 — Remote (US)   
 3  Seagate Technology   Generative AI & Machine Learning Engineer Intern   
 4              OnePay                          Software Engineer, Intern   
 5           Honeywell  Software Engineer & Computer Science - Recent ...   
 6        Tessera Labs                   Backend Software Engineer Intern   
 7              Stripe                        Software Engineer, New Grad   
 8              Zillow      Machine Learning Engineer, Zillow Shopping AI   
 9             Truveta        Software Engineer – Front End (Entry Level)   
 
                                          description  \
 0  About the job\nJob Posting - Salary Range: See...   
 1  About the job\nJob Description\n\

In [135]:
# Open LinkedIn session
driver = open_linkedin_session()

if driver:
    print("\nSession is active! You can now interact with LinkedIn.")
    print(f"Current page: {driver.current_url}")
else:
    print("\nFailed to establish LinkedIn session.")

job_title = "SWE"
job_location = "Seattle"
navigate_to_jobs_page(driver)
find_and_fill_search_boxes(driver, job_title, job_location)

out_file = job_title.replace(" ", "_") + "_" + job_location.replace(" ", "_") + "_jobs.csv"
df, success, errors = extract_linkedin_jobs(driver, max_jobs=20, output_file=out_file)

Loading credentials from environment variables...
Credentials loaded.
Setting up Chrome driver...
Attempting to login to LinkedIn...
Navigating to LinkedIn login page...
Waiting for login form to load...
Entering email...
Entering password...
Clicking sign in button...
Waiting for login to complete...
Login successful - redirected to authenticated page!

Successfully logged into LinkedIn!
Current URL: https://www.linkedin.com/feed/


Session is active! You can now interact with LinkedIn.
Current page: https://www.linkedin.com/feed/
Navigating to LinkedIn Jobs page...
Jobs page loaded successfully!

Filling search boxes...
  Job query: 'SWE'
  Location query: 'Seattle'
  Pressing Enter to submit search...
  ✓ Search submitted successfully!
Switching to jobs iframe...

Found 0 job cards. Extracting 0 jobs...
No jobs extracted!


In [32]:
def save_page_source(driver, filename="linkedin_jobs_page.html"):
    """
    Save page source to file and extract all input element details for debugging.
    
    Args:
        driver: Selenium WebDriver instance
        filename (str): Name of file to save HTML to
        
    Returns:
        str: Filename where page source was saved
    """
    try:
        # Get page source
        page_source = driver.page_source
        
        # Save to file
        with open(filename, 'w', encoding='utf-8') as f:
            f.write(page_source)
        print(f"\n✓ Page source saved to: {filename}")
        
        # Find all input elements
        input_elements = driver.find_elements(By.TAG_NAME, "input")
        
        print(f"\n{'='*80}")
        print(f"Found {len(input_elements)} input elements on the page:")
        print(f"{'='*80}\n")
        
        # Extract and print details of each input
        for i, element in enumerate(input_elements, 1):
            try:
                elem_id = element.get_attribute("id") or "(no id)"
                elem_class = element.get_attribute("class") or "(no class)"
                elem_placeholder = element.get_attribute("placeholder") or "(no placeholder)"
                elem_aria_label = element.get_attribute("aria-label") or "(no aria-label)"
                elem_type = element.get_attribute("type") or "(no type)"
                elem_name = element.get_attribute("name") or "(no name)"
                
                print(f"Input #{i}:")
                print(f"  ID:           {elem_id}")
                print(f"  Class:        {elem_class}")
                print(f"  Type:         {elem_type}")
                print(f"  Name:         {elem_name}")
                print(f"  Placeholder:  {elem_placeholder}")
                print(f"  Aria-label:   {elem_aria_label}")
                print()
                
            except Exception as e:
                print(f"  Error reading element {i}: {e}\n")
        
        # Also save input details to a separate file
        details_filename = filename.replace('.html', '_inputs.txt')
        with open(details_filename, 'w', encoding='utf-8') as f:
            f.write(f"Input Elements Analysis\n")
            f.write(f"{'='*80}\n\n")
            
            for i, element in enumerate(input_elements, 1):
                try:
                    f.write(f"Input #{i}:\n")
                    f.write(f"  ID:           {element.get_attribute('id')}\n")
                    f.write(f"  Class:        {element.get_attribute('class')}\n")
                    f.write(f"  Type:         {element.get_attribute('type')}\n")
                    f.write(f"  Name:         {element.get_attribute('name')}\n")
                    f.write(f"  Placeholder:  {element.get_attribute('placeholder')}\n")
                    f.write(f"  Aria-label:   {element.get_attribute('aria-label')}\n")
                    f.write(f"\n")
                except Exception as e:
                    f.write(f"  Error: {e}\n\n")
        
        print(f"✓ Input element details saved to: {details_filename}")
        
        return filename
        
    except Exception as e:
        print(f"Error saving page source: {e}")
        return None

In [33]:
def fill_linkedin_job_search(driver, job_query="abc", location_query="xyz"):
    """
    Main workflow function to navigate to Jobs page and fill search boxes.
    
    Args:
        driver: Selenium WebDriver instance
        job_query (str): Job title/keyword to search for (default: "abc")
        location_query (str): Location to search in (default: "xyz")
        
    Returns:
        bool: True if successful, False otherwise
    """
    print("\n" + "="*80)
    print("LINKEDIN JOB SEARCH WORKFLOW")
    print("="*80 + "\n")
    
    # Step 1: Navigate to jobs page
    if not navigate_to_jobs_page(driver):
        print("\n❌ Failed to navigate to jobs page")
        return False
    
    random_delay()  # Give page time to fully render
    
    # Step 2: Try to find and fill search boxes
    success, message = find_and_fill_search_boxes(driver, job_query, location_query)
    
    if success:
        print(f"\n{'='*80}")
        print(f"✅ SUCCESS!")
        print(f"{'='*80}")
        print(f"Filled job search with '{job_query}' and location '{location_query}'")
        print(f"{'='*80}\n")
        return True
    else:
        print(f"\n{'='*80}")
        print(f"❌ FAILED: {message}")
        print(f"{'='*80}\n")
        
        # Save page source for debugging
        print("Saving page source for debugging...")
        filename = save_page_source(driver)
        
        if filename:
            print(f"\n{'='*80}")
            print("Page source and input element details saved!")
            print("Check the files to debug selector issues.")
            print(f"{'='*80}\n")
        
        return False

## Step 8: Test LinkedIn Job Search

Run these cells to test the job search functionality.

## Step 9: Additional LinkedIn Interactions (Examples)

Once logged in, you can perform various actions. Here are some examples:

In [34]:
# Test 2: Real job search example
# Try searching for actual jobs

if driver:
    success = fill_linkedin_job_search(
        driver, 
        job_query="Software Engineer", 
        location_query="San Francisco, CA"
    )
    
    print(f"\nSearch completed. Browser is still open for you to see the results.")


LINKEDIN JOB SEARCH WORKFLOW

Navigating to LinkedIn Jobs page...
Jobs page elements not found, but continuing...
Searching for job search box...
Searching for location box...
  ✓ Found location box using: input[placeholder*='City, state']

❌ FAILED: Job search box not found

Saving page source for debugging...

✓ Page source saved to: linkedin_jobs_page.html

Found 2 input elements on the page:

Input #1:
  ID:           :rf:
  Class:        _0ef07236 f6e1d228 cb849684 _90d0b056 c3130085 _7c8e83a5 d80883f4 _5037e19e f058a54c _3ab29e8b adeacb86 _51681c09 e3b4e651 _97bba236 _675760bf e1a71684 _5799a0ba fa2592ae fb86d272 b2618f51 c72a9ef2 _0b692114 _78a69279 _19cb98d9 _01eb8c16 _26c833ed _6093e1cd _430f076b bd361404 _5881c5d5
  Type:         text
  Name:         (no name)
  Placeholder:  Title, skill or Company
  Aria-label:   (no aria-label)

Input #2:
  ID:           :rh:
  Class:        _0ef07236 f6e1d228 cb849684 _90d0b056 c3130085 _7c8e83a5 d80883f4 _5037e19e f058a54c _3ab29e8b ade

In [ ]:
# Test 1: Basic test with default values
# This will search for "abc" and location "xyz"

if driver:
    success = fill_linkedin_job_search(driver, "abc", "xyz")
    
    if success:
        print("✅ Test passed! Search boxes filled successfully.")
    else:
        print("⚠️ Test failed. Check the saved files for debugging.")

## Step 10: Close the Browser

Run this cell when you're done to close the browser and clean up.

## Notes and Tips

1. **Security**: Never commit credentials to version control. Use `.env` files and add them to `.gitignore`.

2. **Rate Limiting**: LinkedIn may detect and block automated access. Use delays between actions to appear more human-like.

3. **CAPTCHA**: If LinkedIn presents a CAPTCHA, you'll need to solve it manually.

4. **Two-Factor Authentication**: If your account has 2FA enabled, you may need to handle additional verification steps.

5. **Terms of Service**: Make sure your use case complies with LinkedIn's Terms of Service.

6. **Error Handling**: The functions include basic error handling, but you may need to add more depending on your use case.

7. **Dynamic Selectors**: LinkedIn uses dynamic IDs (e.g., `ember123`). The functions include multiple selector strategies to handle this.

8. **Debugging**: If search boxes aren't found, the `save_page_source()` function will save the HTML and list all input elements to help you debug.

## Step 7: Interact with LinkedIn (Examples)

Once logged in, you can perform various actions. Here are some examples:

In [ ]:
# Example: Navigate to your profile
if driver:
    driver.get("https://www.linkedin.com/in/me/")
    time.sleep(2)
    print(f"Navigated to: {driver.current_url}")

In [ ]:
# Example: Search for jobs
if driver:
    driver.get("https://www.linkedin.com/jobs/")
    time.sleep(2)
    print(f"Navigated to: {driver.current_url}")

In [ ]:
# Example: Get page title
if driver:
    print(f"Page title: {driver.title}")

## Notes and Tips

1. **Security**: Never commit credentials to version control. Use `.env` files and add them to `.gitignore`.

2. **Rate Limiting**: LinkedIn may detect and block automated access. Use delays between actions to appear more human-like.

3. **CAPTCHA**: If LinkedIn presents a CAPTCHA, you'll need to solve it manually.

4. **Two-Factor Authentication**: If your account has 2FA enabled, you may need to handle additional verification steps.

5. **Terms of Service**: Make sure your use case complies with LinkedIn's Terms of Service.

6. **Error Handling**: The functions include basic error handling, but you may need to add more depending on your use case.